# V-JEPA Segmentation — Colab Demo

This notebook installs your repo, downloads weights (if URLs are provided), and runs **image** and **video** inference.

**How to use**
1. **Edit the variables** in the next cell: `REPO_URL`, `BRANCH`, `HEAD_URL`, `IMAGE_PATH`, `VIDEO_PATH`.
2. Run all cells (Runtime → Run all). If you leave `HEAD_URL` empty, you can upload your head checkpoint manually to `ckpts/` via the Colab file browser.

*Tip:* In Colab, choose **Runtime → Change runtime type → GPU** for speed.

In [0]:
# ⬇️ EDIT THESE ⬇️
REPO_URL = "https://github.com/YOUR_USERNAME/vjepa_segmentation.git"  # <-- change to your repo
BRANCH   = "main"
HEAD_URL = ""  # optional: GitHub Release asset for ckpts/head_linear_fpn_best.pt

# sample inputs relative to repo root (replace if needed)
IMAGE_PATH = "samples/person.jpg"
VIDEO_PATH = "samples/clip.mp4"

print("REPO_URL:", REPO_URL)
print("BRANCH:", BRANCH)
print("HEAD_URL set:", bool(HEAD_URL))
print("IMAGE_PATH:", IMAGE_PATH)
print("VIDEO_PATH:", VIDEO_PATH)

In [0]:
import sys, subprocess, os
!nvidia-smi -L || echo "No GPU (that's ok, but slower)"
!python --version
!pip -q install -U pip
# core libs we rely on in your repo
!pip -q install timm einops opencv-python

In [0]:
import os
from pathlib import Path

repo_dir = Path("vjepa_segmentation")
if not repo_dir.exists():
    !git clone -b "$BRANCH" "$REPO_URL" vjepa_segmentation
%cd vjepa_segmentation

# Install project (best-effort on requirements)
!pip -q install -r requirements.txt || true
!pip -q install -e .

# Folders for outputs
os.makedirs("ckpts", exist_ok=True)
os.makedirs("media", exist_ok=True)

In [0]:
import os, urllib.request

# Backbone (V-JEPA2 ViT-L/16). Official link from Meta's README.
vjepa_pt = "ckpts/vjepa2_vitl_256.pt"
if not os.path.exists(vjepa_pt):
    url = "https://dl.fbaipublicfiles.com/vjepa2/vitl.pt"
    print("Downloading backbone ->", vjepa_pt)
    urllib.request.urlretrieve(url, vjepa_pt)
else:
    print("Found backbone:", vjepa_pt)

# Head checkpoint (your small segmentation head). Optional auto-download.
head_pt = "ckpts/head_linear_fpn_best.pt"
if os.path.exists(head_pt):
    print("Found head:", head_pt)
elif HEAD_URL:
    print("Downloading head ->", head_pt)
    urllib.request.urlretrieve(HEAD_URL, head_pt)
else:
    print("No head checkpoint found and HEAD_URL not set.\n"
          "Upload ckpts/head_linear_fpn_best.pt via the Colab file browser if you want real inference.\n"
          "Otherwise, the notebook will run a dummy smoke test.")

In [0]:
import os, shlex, subprocess

def run(cmd_list):
    print("RUN:", " ".join(shlex.quote(c) for c in cmd_list))
    return subprocess.run(cmd_list, check=False)

img_exists = os.path.exists(IMAGE_PATH)
have_head = os.path.exists("ckpts/head_linear_fpn_best.pt")
have_back = os.path.exists("ckpts/vjepa2_vitl_256.pt")

if not img_exists:
    print("Image not found:", IMAGE_PATH)
else:
    os.makedirs("media", exist_ok=True)
    if have_head and have_back:
        cmd = [
            "python","-m","vjepa_seg.inference",
            "--config","configs/cihp_linear_fpn.yaml",
            "--image", IMAGE_PATH,
            "--out","media/demo_image_overlay.jpg",
            "--ckpt","ckpts/head_linear_fpn_best.pt",
            "--vjepa_ckpt","ckpts/vjepa2_vitl_256.pt",
        ]
    else:
        cmd = [
            "python","-m","vjepa_seg.inference",
            "--config","configs/cihp_linear_fpn.yaml",
            "--image", IMAGE_PATH,
            "--out","media/smoke_out.png",
            "--ckpt","",
            "--dummy_backbone","1",
        ]
    run(cmd)

from IPython.display import Image, display
if os.path.exists("media/demo_image_overlay.jpg"):
    display(Image(filename="media/demo_image_overlay.jpg"))
elif os.path.exists("media/smoke_out.png"):
    display(Image(filename="media/smoke_out.png"))

In [0]:
import os, shlex, subprocess
from IPython.display import HTML, display

vid_exists = os.path.exists(VIDEO_PATH)
have_head = os.path.exists("ckpts/head_linear_fpn_best.pt")
have_back = os.path.exists("ckpts/vjepa2_vitl_256.pt")

def run(cmd_list):
    print("RUN:", " ".join(shlex.quote(c) for c in cmd_list))
    return subprocess.run(cmd_list, check=False)

if not vid_exists:
    print("Video not found:", VIDEO_PATH)
else:
    if have_head and have_back:
        cmd = [
            "python","-m","vjepa_seg.inference",
            "--config","configs/cihp_linear_fpn.yaml",
            "--video", VIDEO_PATH,
            "--out","out.mp4",
            "--ema","0.6",
            "--ckpt","ckpts/head_linear_fpn_best.pt",
            "--vjepa_ckpt","ckpts/vjepa2_vitl_256.pt",
        ]
    else:
        cmd = [
            "python","-m","vjepa_seg.inference",
            "--config","configs/cihp_linear_fpn.yaml",
            "--video", VIDEO_PATH,
            "--out","out.mp4",
            "--ema","0.0",
            "--ckpt","",
            "--dummy_backbone","1",
        ]
    run(cmd)
    
    # Make a small GIF preview for easy embedding in README
    if os.path.exists("out.mp4"):
        !ffmpeg -y -i out.mp4 -vf "fps=15,scale=512:-1:flags=lanczos" -t 4 media/demo_video.gif
        if os.path.exists("media/demo_video.gif"):
            display(HTML('<img src="media/demo_video.gif" />'))

## ✅ Done
- Image result saved to `media/demo_image_overlay.jpg` (or `media/smoke_out.png` for the dummy test)
- Video GIF preview saved to `media/demo_video.gif` (if video ran)

**Add these to your README:**
```markdown
![Demo overlay](media/demo_image_overlay.jpg)
![Demo video](media/demo_video.gif)
```

If you want the notebook badge in your README:
```markdown
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
  https://colab.research.google.com/github/YOUR_USERNAME/vjepa_segmentation/blob/main/notebooks/01_colab_demo.ipynb)
```

*(Update the repository path to match your username/branch.)*